In [23]:
!pip install openai==1.12.0 

In [2]:
import mlflow
import openai
import os
import pandas as pd
from getpass import getpass

/home/olonok/.local/lib/python3.11/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [4]:
mlflow.set_tracking_uri("http://localhost:5000") 

In [5]:
experiment_name = "openai-evaluate"
try:
    mlflow.create_experiment(experiment_name)
except:
    print("experiment exists")
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///home/olonok/mlflow/mlruns/3', creation_time=1715346281897, experiment_id='3', last_update_time=1715346281897, lifecycle_stage='active', name='openai-evaluate', tags={}>

In [11]:
OPENAI_API_KEY = getpass()

 ········


In [12]:
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [13]:
eval_data = pd.DataFrame(
    {
        "inputs": [
            "What is MLflow?",
            "What is Spark?",
        ],
        "ground_truth": [
            "MLflow is an open-source platform for managing the end-to-end machine learning (ML) "
            "lifecycle. It was developed by Databricks, a company that specializes in big data and "
            "machine learning solutions. MLflow is designed to address the challenges that data "
            "scientists and machine learning engineers face when developing, training, and deploying "
            "machine learning models.",
            "Apache Spark is an open-source, distributed computing system designed for big data "
            "processing and analytics. It was developed in response to limitations of the Hadoop "
            "MapReduce computing model, offering improvements in speed and ease of use. Spark "
            "provides libraries for various tasks such as data ingestion, processing, and analysis "
            "through its components like Spark SQL for structured data, Spark Streaming for "
            "real-time data processing, and MLlib for machine learning tasks",
        ],
    }
)

In [18]:
client = openai.OpenAI()

In [19]:
client.chat.completions.create

<bound method Completions.create of <openai.resources.chat.completions.Completions object at 0x7f575dbff050>>

In [22]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    # Wrap "gpt-4" as an MLflow model.
    logged_model_info = mlflow.openai.log_model(
        model="gpt-4",
        task=client.chat.completions,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )

    # Use predefined question-answering metrics to evaluate our model.
    results = mlflow.evaluate(
        logged_model_info.model_uri,
        eval_data,
        targets="ground_truth",
        model_type="question-answering",
    )
    print(f"See aggregated evaluation results below: \n{results.metrics}")

    # Evaluation result for each data record is available in `results.tables`.
    eval_table = results.tables["eval_results_table"]
    print(f"See evaluation table below: \n{eval_table}")

MlflowException: Unsupported task type: <class 'openai.resources.chat.completions.Completions'>